In [1]:
# Implementation based on the following paper:

# https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [2]:
# Key concept: Words are embedded in an n-dimensional embedding space. The MLP learns to organize this space in a way that forms understanding of
#              semantics. The goal is to give similar embeddings to similar words. Each embedding dimensions will encode the model's compressed
#              understanding. This allows the model to transfer knowledge and generalize to novel data.

In [3]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

In [4]:
words = open('names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [6]:
# Build the vocabulary
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)} # {} to create a dictionary, i+1 so the starting index is 1, enumerate returns an iterable of tuples
print(f'{stoi = }')

# Syntax is: {key_expression: value_expression for item in iterable}

stoi['.'] = 0 # Add start/end token with 0 index
itos = {i:s for s,i in stoi.items()}
print(f'{itos = }')

stoi = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
itos = {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
list(enumerate(chars))

[(0, 'a'),
 (1, 'b'),
 (2, 'c'),
 (3, 'd'),
 (4, 'e'),
 (5, 'f'),
 (6, 'g'),
 (7, 'h'),
 (8, 'i'),
 (9, 'j'),
 (10, 'k'),
 (11, 'l'),
 (12, 'm'),
 (13, 'n'),
 (14, 'o'),
 (15, 'p'),
 (16, 'q'),
 (17, 'r'),
 (18, 's'),
 (19, 't'),
 (20, 'u'),
 (21, 'v'),
 (22, 'w'),
 (23, 'x'),
 (24, 'y'),
 (25, 'z')]

In [37]:
# Build the dataset

block_size = 3 # Context length
X, Y = [], []

for w in words: # run with words[:5] if checking
    # print(w)
    context = [0] * block_size # Multiplication of a list with a positive integer replicates the contents of the list that many times. 
                               # here [0, 0, 0]
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), ' ----> ', itos[ix])
        
        context = context[1:] + [ix] # Crop and append. This essentially shifts the context window to the right.

X = torch.tensor(X)
Y = torch.tensor(Y) 

In [9]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [38]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [11]:
# Create the lookup table "C" of embeddings per character
g = torch.Generator().manual_seed(2147483647) 
C = torch.randn((27,2), generator=g) # 27 characters embedded in a 2-dimensional space. Returns a tensor.
print(f'{C.shape = }, \n{C.dtype = }, \n{C = }')

C.shape = torch.Size([27, 2]), 
C.dtype = torch.float32, 
C = tensor([[ 1.5674, -0.2373],
        [-0.0274, -1.1008],
        [ 0.2859, -0.0296],
        [-1.5471,  0.6049],
        [ 0.0791,  0.9046],
        [-0.4713,  0.7868],
        [-0.3284, -0.4330],
        [ 1.3729,  2.9334],
        [ 1.5618, -1.6261],
        [ 0.6772, -0.8404],
        [ 0.9849, -0.1484],
        [-1.4795,  0.4483],
        [-0.0707,  2.4968],
        [ 2.4448, -0.6701],
        [-1.2199,  0.3031],
        [-1.0725,  0.7276],
        [ 0.0511,  1.3095],
        [-0.8022, -0.8504],
        [-1.8068,  1.2523],
        [ 0.1476, -1.0006],
        [-0.5030, -1.0660],
        [ 0.8480,  2.0275],
        [-0.1158, -1.2078],
        [-1.0406, -1.5367],
        [-0.5132,  0.2961],
        [-1.4904, -0.2838],
        [ 0.2569,  0.2130]])


In [12]:
C[X].shape # We can index into the tensor C using another tensor.

# The shape is (sample x character x embedding)
# Remember 1 "sample" is made of 3 characters (the preceding 3, to be exact) at that is why the first two dimensions are 32 x 3,
# and "embedding " consists of 2 values, which is why the last dimension is 2.

torch.Size([32, 3, 2])

In [13]:
# Explanation of pytorch advanced indexing

# What Happens in C[X]?
# When you do C[X], you're not directly indexing C as if C were 3D. Instead, you're using the advanced indexing feature in PyTorch, 
# where the tensor X acts as a lookup table for rows in C. Here's how it works:

# Indexing Mechanism:
# X specifies which rows of C to select.
# Each ENTRY of X is an index that tells PyTorch which row of C to pick.

print(f'{C[0] = }') # This selects the first row of C
print(f'{C[X][0,:,0] = }') # This grabs the first embedding for each character in the first sample (sample = context, made of 3 characters)
                           # So the 3rd dimension is the useful one which contains the embeddings for each character


C[0] = tensor([ 1.5674, -0.2373])
C[X][0,:,0] = tensor([1.5674, 1.5674, 1.5674])


In [14]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [15]:
# Build hidden layer

# Matrix of weights

W1 = torch.randn((6, 100), generator=g)  # The NN takes the previous 3 characters and predicts the next one,
                            # and each character was embedded using 2 values (2D embedding space).
                            # This means that the NN takes (3 characters * 2 embeddings/character = 6 values),
                            # and we set the layer to have 100 neurons.
                        
b1 = torch.randn(100, generator=g)       # Make 100 biases, 1 per neuron.

In [16]:
emb

tensor([[[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [ 1.5674, -0.2373]],

        [[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [-0.4713,  0.7868]],

        [[ 1.5674, -0.2373],
         [-0.4713,  0.7868],
         [ 2.4448, -0.6701]],

        [[-0.4713,  0.7868],
         [ 2.4448, -0.6701],
         [ 2.4448, -0.6701]],

        [[ 2.4448, -0.6701],
         [ 2.4448, -0.6701],
         [-0.0274, -1.1008]],

        [[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [ 1.5674, -0.2373]],

        [[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [-1.0725,  0.7276]],

        [[ 1.5674, -0.2373],
         [-1.0725,  0.7276],
         [-0.0707,  2.4968]],

        [[-1.0725,  0.7276],
         [-0.0707,  2.4968],
         [ 0.6772, -0.8404]],

        [[-0.0707,  2.4968],
         [ 0.6772, -0.8404],
         [-0.1158, -1.2078]],

        [[ 0.6772, -0.8404],
         [-0.1158, -1.2078],
         [ 0.6772, -0.8404]],

        [[-0.1158, -1

In [17]:
# Now, the layer has to perform a matrix multiplication to operate on the inputs:

# Something like emb @ W1 + b1

# But this can't be done because the dimensions of emb and W1 are not compatible. Therefore, we need to reshape emb e.g. through concatenation.


torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1)    # This grabs the embeddings of the 1st char, 2nd char, and 3rd char
                                                            # and adds them as columns (dim 1)

# The problem is that this code does not generalize because we have to manually list the tensors in the list that is passed to cat.

tensor([[ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.4713,  0.7868],
        [ 1.5674, -0.2373, -0.4713,  0.7868,  2.4448, -0.6701],
        [-0.4713,  0.7868,  2.4448, -0.6701,  2.4448, -0.6701],
        [ 2.4448, -0.6701,  2.4448, -0.6701, -0.0274, -1.1008],
        [ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -1.0725,  0.7276],
        [ 1.5674, -0.2373, -1.0725,  0.7276, -0.0707,  2.4968],
        [-1.0725,  0.7276, -0.0707,  2.4968,  0.6772, -0.8404],
        [-0.0707,  2.4968,  0.6772, -0.8404, -0.1158, -1.2078],
        [ 0.6772, -0.8404, -0.1158, -1.2078,  0.6772, -0.8404],
        [-0.1158, -1.2078,  0.6772, -0.8404, -0.0274, -1.1008],
        [ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.0274, -1.1008],
        [ 1.5674, -0.2373, -0.0274, -1.1008, -0.1158, -1.2078],
        [-0.0274, -1.1008, -0.1158, -1.2

In [18]:
# Better way with torch.unbind(emb, 1), which is equal to [emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]] but generalizes.

torch.cat(torch.unbind(emb, 1), 1)

tensor([[ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.4713,  0.7868],
        [ 1.5674, -0.2373, -0.4713,  0.7868,  2.4448, -0.6701],
        [-0.4713,  0.7868,  2.4448, -0.6701,  2.4448, -0.6701],
        [ 2.4448, -0.6701,  2.4448, -0.6701, -0.0274, -1.1008],
        [ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -1.0725,  0.7276],
        [ 1.5674, -0.2373, -1.0725,  0.7276, -0.0707,  2.4968],
        [-1.0725,  0.7276, -0.0707,  2.4968,  0.6772, -0.8404],
        [-0.0707,  2.4968,  0.6772, -0.8404, -0.1158, -1.2078],
        [ 0.6772, -0.8404, -0.1158, -1.2078,  0.6772, -0.8404],
        [-0.1158, -1.2078,  0.6772, -0.8404, -0.0274, -1.1008],
        [ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.0274, -1.1008],
        [ 1.5674, -0.2373, -0.0274, -1.1008, -0.1158, -1.2078],
        [-0.0274, -1.1008, -0.1158, -1.2

In [19]:
# But there is actually a more efficient way.

# Proof:

a = torch.arange(18)
print(f'{a = }, \n{a.shape = }, \n\n{a.storage() = }') # Note: a.storage() returns a TypedStorage object, which is deprecated. Use .untyped_storage()

# Note: In computer memory the data of a tensor is always represented as a 1D vector!
#       Pytorch then interprets this number sequence as a tensor of specific dimensions.

a = tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]), 
a.shape = torch.Size([18]), 

a.storage() =  0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]


C:\Users\emman\AppData\Local\Temp\ipykernel_13288\592617819.py:6: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  print(f'{a = }, \n{a.shape = }, \n\n{a.storage() = }') # Note: a.storage() returns a TypedStorage object, which is deprecated. Use .untyped_storage()


In [20]:
# On the other hand,

print(f'{a.view(3,3,2) = }') # a.view(9, 9), a.view(6, 3), etc.

#  .view() is therefore more efficient than .cat() in PyTorch because .view() only alters the shape of the tensor by modifying its metadata, 
#  without changing the underlying data. This operation is computationally inexpensive since no new memory allocation occurs. 
#  In contrast, .cat() creates a new tensor by concatenating existing tensors, which involves copying data and allocating new memory, 
#  making it more resource-intensive.

# So instead we can do:

print(f'\n{emb.view(32, 6) = }')


a.view(3,3,2) = tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

emb.view(32, 6) = tensor([[ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.4713,  0.7868],
        [ 1.5674, -0.2373, -0.4713,  0.7868,  2.4448, -0.6701],
        [-0.4713,  0.7868,  2.4448, -0.6701,  2.4448, -0.6701],
        [ 2.4448, -0.6701,  2.4448, -0.6701, -0.0274, -1.1008],
        [ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -1.0725,  0.7276],
        [ 1.5674, -0.2373, -1.0725,  0.7276, -0.0707,  2.4968],
        [-1.0725,  0.7276, -0.0707,  2.4968,  0.6772, -0.8404],
        [-0.0707,  2.4968,  0.6772, -0.8404, -0.1158, -1.2078],
        [ 0.6772, -0.8404, -0.1158, -1.2078,  0.6772, -0.8404],
        [-0.1158, -1.2078,  0.6772, -0.8404, -0.0274, -1.1008],
        [ 1.5674, -0

In [21]:
# Perform hidden layer operation

# emb.view(32, 6) @ W1 + b1
emb.view(emb.shape[0], 6) @ W1 + b1     # Even better so dims are not hardcoded.
                                        # emb.shape[0] selects the first dimension given by emb.shape.
                                        # Can also use -1 as the first dim so Pytorch derives it itself, knowing that the number of elements has to
                                        # be equal to the original tensor's.

# Corresponds to the logits of the hidden layer, pre-activation function

tensor([[-1.6952e+00,  8.5502e+00,  1.6284e+00,  ...,  2.2642e+00,
         -1.9505e-01,  1.8469e+00],
        [ 2.8741e-01,  4.3343e+00,  1.0142e+00,  ...,  2.8221e+00,
          3.9128e+00,  3.4733e+00],
        [-3.1026e+00,  9.9601e+00, -1.3306e+00,  ..., -5.7069e-01,
         -5.9107e+00, -6.9120e-03],
        ...,
        [-4.3248e+00,  7.4938e+00, -1.6386e+00,  ..., -5.1557e+00,
         -3.3276e+00, -3.2464e+00],
        [-1.4951e+00,  5.6195e+00,  2.5079e+00,  ..., -1.0607e+00,
         -5.2543e-01,  3.4893e+00],
        [-1.4982e+00,  8.5941e+00,  1.8897e+00,  ...,  2.4983e+00,
          6.9596e+00,  2.6822e+00]])

In [22]:
# Calculate activations

h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

print(f'{h = }, \n\n{h.shape = }')

h = tensor([[-0.9348,  1.0000,  0.9258,  ...,  0.9786, -0.1926,  0.9515],
        [ 0.2797,  0.9997,  0.7675,  ...,  0.9929,  0.9992,  0.9981],
        [-0.9960,  1.0000, -0.8694,  ..., -0.5159, -1.0000, -0.0069],
        ...,
        [-0.9996,  1.0000, -0.9273,  ..., -0.9999, -0.9974, -0.9970],
        [-0.9043,  1.0000,  0.9868,  ..., -0.7859, -0.4819,  0.9981],
        [-0.9048,  1.0000,  0.9553,  ...,  0.9866,  1.0000,  0.9907]]), 

h.shape = torch.Size([32, 100])


In [23]:
# Broadcasting correctness check:

print(f'{(emb.view(emb.shape[0], 6) @ W1).shape = }')
print(f'\n{b1.shape = }')

# So internally the broadcasting will

# 32, 100
#     100

# first align the dimensions from the right,

# 32, 100
#  1, 100

# then add a dimension made of 1s entries (this turns b1 into a row vector),
# and then this row vector will be copied vertically to match the 32, 100 dim necessary for the addition

# Conclusion: broadcasting in this case achieves the desired operation as the element-wise addition is performed with the same values for every row,
# meaning each neuron (dim 1) adds the same bias to each sample.

(emb.view(emb.shape[0], 6) @ W1).shape = torch.Size([32, 100])

b1.shape = torch.Size([100])


In [24]:
# Make output layer

W2 = torch.randn((100, 27), generator=g) # 100 input values from the previous layer (1 per neuron), and 27 outputs (a probability distribution over 27 characters)
b2 = torch.randn(27, generator=g)

In [25]:
logits = h @ W2 + b2
print(f'{logits.shape = }')

logits.shape = torch.Size([32, 27])


In [26]:
# Implement softmax fxn

counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True) # Reminder: summing along dim 1 means summing 1 row across all it's columns (so it's a row sum)
                                            # Reminder: keepdim=True necessary otherwise 1 dimension is collapsed, messing with broadcasting

In [27]:
Y.shape # 1 label character per training sample

torch.Size([32])

In [28]:
# Take the probabilities assigned by the NN to the correct characters (Y) following each sample sequence.

prob[torch.arange(32), Y] # This provides two iterators of indices (tensors), since Pytorch supports advanced indexing there is no need to zip

tensor([1.5213e-14, 1.2830e-12, 1.9647e-08, 3.1758e-10, 5.6763e-12, 1.0823e-10,
        1.8821e-14, 1.1087e-08, 1.6134e-09, 2.1917e-03, 5.3863e-08, 3.1970e-04,
        2.0283e-10, 3.5709e-11, 6.2336e-07, 5.1704e-07, 1.4206e-01, 9.5657e-09,
        2.0670e-09, 2.5181e-02, 7.6846e-05, 2.8706e-12, 1.6961e-09, 5.6464e-15,
        4.4656e-03, 2.6851e-09, 3.5864e-05, 2.3389e-04, 1.6890e-09, 9.5614e-01,
        9.7404e-10, 2.1230e-12])

In [29]:
# Check number of total parameters

parameters = [C, W1, b1, W2, b2]
sum(p.nelement() for p in parameters)   

3481

In [30]:
# Loss function - Negative log likelihood

# Manual NLL:

# loss = -prob[torch.arange(32), Y].log().mean() # 

loss = F.cross_entropy(logits, Y)   # Cross entropy loss is just NLL + built in softmax,
                                    # but more efficient since Pytorch uses a fused kernels to do all the operations and so uses less memory and time.
                                    # Additionally, backprop becomes more efficient because the derivatives of fused operations are analytically simpler

print(f'{loss = }')

# Also, manual implementation runs into this problem:
 
# logits = torch.tensor([-100, -3, 0, 100]) # large positive number exceeds the range of the floating point datatype, resulting in "inf"
# counts = logits.exp()
# probs = counts / counts.sum()
# probs

# The way Pytorch solves this internally is that it scales the logits down by subtracting the highest logit, since this does not change the
# probabilities

loss = tensor(17.7697)


## Full implementation

In [39]:
print(f'{X.shape = }, \n{Y.shape = }') # Check data dims

X.shape = torch.Size([228146, 3]), 
Y.shape = torch.Size([228146])


In [45]:
# Parameters

g = torch.Generator().manual_seed(2147483647)  # for reproducibility
C = torch.randn(27, 2, generator=g)
W1 = torch.randn(6, 100, generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn(100, 27, generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

sum(p.nelement() for p in parameters)  # number of parameters in total

3481

In [46]:
for p in parameters: # Pytorch requirement
    p.requires_grad = True

In [42]:
for _ in range(1000):

    # Minibatch construction
    ix = torch.randint(0, X.shape[0], (32,)) # Last parameter specifies the dimensions of the returned tensor, here (32,) means a 1D vector

    # Forward pass
    emb = C[X[ix]] # (32, 3, 2), indexing into X, which is the full training set, and grabbing 32 random samples
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # (32, 100)
    logits = h @ W2 + b2  # (32, 27)
    loss = F.cross_entropy(logits, Y[ix])
    # print(loss.item())
    
    # Backward pass
    for p in parameters:    
        p.grad = None   # IMPORTANT Note: Need to reset the gradients every training run.
                        # Otherwise the model would think the error is much larger than it actually is, 
                        # because it's adding errors from previous batches or iterations together, which is not relevant to the current iteration's
                        # loss!
    loss.backward()

    # Update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item()) # final batch loss

19.444650650024414
16.026756286621094
19.595340728759766
14.392146110534668
16.09246253967285
12.661137580871582
13.228506088256836
11.690152168273926
12.289734840393066
10.544520378112793
9.81041431427002
9.253498077392578
10.30218505859375
9.657097816467285
10.381988525390625
8.897500038146973
8.594795227050781
8.87955093383789
6.760862350463867
8.90519905090332
9.463692665100098
7.72509765625
6.688868522644043
6.6442131996154785
6.783695220947266
8.885218620300293
7.291264533996582
6.676371097564697
7.230817794799805
6.5191826820373535
7.807373523712158
7.735731601715088
8.524930953979492
5.371830463409424
7.295154571533203
6.7521514892578125
6.151376247406006
7.431678771972656
5.562653541564941
7.038995265960693
5.1658830642700195
4.6005449295043945
6.156202793121338
5.665262222290039
5.118688583374023
4.969459533691406
7.730938911437988
4.054111003875732
4.814337253570557
7.172355651855469
6.0982890129089355
4.236999034881592
5.804749011993408
5.7193379402160645
5.803781986236572


In [44]:
# Final forward pass to check total loss:

emb = C[X[ix]] # (32, 3, 2), indexing into X, which is the full training set, and grabbing 32 random samples
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2  # (32, 27)
loss = F.cross_entropy(logits, Y[ix])
loss

tensor(2.2519, grad_fn=<NllLossBackward0>)